<a href="https://colab.research.google.com/github/jp12AI/3d-vehicle-tracking/blob/master/PyTorch_Model%2B_PID_%2B_MAP_INTERFACE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
##code an e2e PyTorch AI Model in python suitable for self-driving car applications, that takes in input from cameras only 
import torch
import torch.nn as nn
import torch.optim as optim

# Define the neural network architecture
class SelfDrivingCarModel(nn.Module):
    def __init__(self):
        super(SelfDrivingCarModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)  # 3 input channels (RGB), 6 output channels, 5x5 kernel
        self.pool = nn.MaxPool2d(2, 2)  # 2x2 max pooling
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)  # fully-connected layer with 120 output units
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 1)  # output layer with 1 output unit (for steering angle prediction)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)  # flatten the data for the fully-connected layers
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Create an instance of the model and move it to the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SelfDrivingCarModel().to(device)

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Train the model
for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

# Test the model
with torch.no_grad():
    correct = 0
    total = 0
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    print(f'Accuracy of the model on the test set: {100 * correct / total}%')


NameError: ignored

In [ ]:
##Code a PID controller that takes in inputs from self driving model and then outputs to a motor that moves the steering wheel 
import time

class PIDController:
    def __init__(self, kp, ki, kd, max_output, min_output):
        self.kp = kp  # Proportional gain
        self.ki = ki  # Integral gain
        self.kd = kd  # Derivative gain
        self.max_output = max_output  # Maximum output value
        self.min_output = min_output  # Minimum output value
        self.last_error = 0  # Last error value
        self.integral = 0  # Integral of errors over time

    def control(self, error, dt):
        """
        Calculates the control output based on the given error and time delta.
        """
        # Calculate the proportional term
        p = self.kp * error
        
        # Calculate the integral term
        self.integral += error * dt
        i = self.ki * self.integral
        
        # Calculate the derivative term
        d = self.kd * (error - self.last_error) / dt
        
        # Calculate the control output
        output = p + i + d
        
        # Update the last error value
        self.last_error = error
        
        # Bound the output value between the minimum and maximum output values
        if output > self.max_output:
            output = self.max_output
        elif output < self.min_output:
            output = self.min_output
        
        return output

# Create an instance of the PIDController class
pid_controller = PIDController(kp=1.0, ki=0.1, kd=0.01, max_output=1.0, min_output=-1.0)

# Main loop
while True:
    # Get the current error value from the self-driving car model
    error = self_driving_car_model.get_error()

    # Calculate the control output based on the error
    control_output = pid_controller.control(error, dt=0.1)

    # Set the control output to the motor that moves the steering wheel
    steering_motor.set_output(control_output)
    
    time.sleep(0.1)  # Sleep for 0.1 seconds


NameError: ignored

In [ ]:
pip install mapbox

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 132 kB 46.9 MB/s 
     |████████████████████████████████| 79 kB 8.5 MB/s 
     |████████████████████████████████| 10.3 MB 32.4 MB/s 
     |████████████████████████████████| 140 kB 62.5 MB/s 
     |████████████████████████████████| 127 kB 51.4 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
##code a navigation interface using mapbox api suitable for PID self driving cars
import mapbox

# Replace YOUR_ACCESS_TOKEN with your Mapbox access token
mapbox_api = mapbox.Client(access_token="YOUR_ACCESS_TOKEN")

class NavigationInterface:
    def __init__(self, start_lat, start_long, end_lat, end_long):
        # Store start and end coordinates
        self.start_lat = start_lat
        self.start_long = start_long
        self.end_lat = end_lat
        self.end_long = end_long

        # Get route from Mapbox API
        route = mapbox_api.directions([(start_long, start_lat), (end_long, end_lat)])
        self.route_coordinates = route["routes"][0]["geometry"]["coordinates"]
        self.current_index = 0

    def get_next_waypoint(self):
        # Return next waypoint in route
        if self.current_index < len(self.route_coordinates):
            waypoint = self.route_coordinates[self.current_index]
            self.current_index += 1
            return waypoint
        else:
            return None

# Example usage
nav = NavigationInterface(start_lat=37.78, start_long=-122.41, end_lat=37.79, end_long=-122.42)
waypoint = nav.get_next_waypoint()
print(waypoint)  # Output: [-122.41, 37.78]
##This NavigationInterface class uses the Mapbox Directions API to retrieve a route between the start and end coordinates, and stores the route coordinates in a list. The get_next_waypoint function retrieves the next waypoint in the route and increments the current_index variable to keep track of the current position in the route.
##You can then use the waypoint returned by get_next_waypoint as a setpoint for your PID controller to control the self-driving car's navigation.

AttributeError: ignored